---
sidebar_position: 2
---

> You can get this [Notebook](https://github.com/Seeed-Projects/Tutorial-of-AI-Kit-with-Raspberry-Pi-From-Zero-to-Hero/blob/main/articles/Chapter_5-Custom_Model_Development_and_Deployment/Convert_Your_Model.ipynb) on GitHub.

# Step 1: Prepare your environment on your host computer

**📌 Note:：This part of code run on your linux host computer with python3.10**

The Jupyter Notebook right up have a button like ![select kernel](../../pictures/Chapter5/select_kernel.png), then you choose ```Select Another Kernel```, and choose ```Python Environments```, then choose ```Creat Python Environment``` and choose ```Venv```, then choose ```python3.10```.

In [1]:
# Here is my hostcomputer information
# Linux PC 6.8.0-45-generic #45~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Wed Sep 11 15:25:05 UTC 2 x86_64 x86_64 x86_64 GNU/Linux

!uname -a

Linux PC 6.8.0-48-generic #48~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Mon Oct  7 11:24:13 UTC 2 x86_64 x86_64 x86_64 GNU/Linux


In [2]:
# Here is my python version
# Python 3.10.12

!python -V

Python 3.10.12


In [4]:
# Download hailo model zoo to convert onnx model
try:
    ! git clone https://github.com/LJ-Hao/hailo_model_zoo.git
except Exception as e:
    print(f'install error: {e}')

fatal: destination path 'hailo_model_zoo' already exists and is not an empty directory.


## Here we need [Hailo DataFlow Compiler](https://hailo.ai/products/hailo-software/hailo-ai-software-suite/#sw-dc) to convert our model 

### Download from official website

#### 1. Register a [Hailo account](https://hailo.ai/developer-zone/request-access/) 

![register_hailo](../../pictures/Chapter5/register_hailo.png)

#### 2. Install [Hailo DataFlow Compiler](https://hailo.ai/developer-zone/software-downloads/)

![install_hailo](../../pictures/Chapter5/install_compiler.png)

In [6]:
# Install HailoDFC to compile the model

try:
    %pip install ../resource/hailo_dataflow_compiler-3.29.0-py3-none-linux_x86_64.whl -q
    print('install successfully')
except:
    print('install error')

Note: you may need to restart the kernel to use updated packages.
install successfully


In [7]:
# Install hailo model zoo

try:
    ! cd hailo_model_zoo/ && pip install -e . -q
except Exception as e:
    print(f'install error: {e}')

  DEPRECATION: Legacy editable install of hailo-model-zoo==2.13.0 from file:///home/jiahao/Tutorial-of-AI-Kit-with-Raspberry-Pi-From-Zero-to-Hero/articles/Chapter_5-Custom_Model_Development_and_Deployment/hailo_model_zoo (setup.py develop) is deprecated. pip 25.0 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457


In [8]:
# Check if HailoModel Zoo is installed 

! hailomz info mobilenet_v1

<Hailo Model Zoo INFO> Start run for network mobilenet_v1 ...
<Hailo Model Zoo INFO> 
	task:                    classification
	input_shape:             224x224x3
	output_shape:            1001
	operations:              1.14G
	parameters:              4.22M
	framework:               tensorflow
	training_data:           imagenet train
	validation_data:         imagenet val
	eval_metric:             Accuracy (top1)
	full_precision_result:   70.97
	source:                  https://github.com/tensorflow/models/tree/v1.13.0/research/slim
	license_url:             https://github.com/tensorflow/models/blob/master/LICENSE


# Step 2: Using HailoDFC and HailoModelZoo to compile ONNX model to HEF model.

In [9]:
# Copy your onnx model to hailo_model_zoo folder

! cp ../../models/Chapter5/best.onnx ./hailo_model_zoo/

In [ ]:
# This command demonstrates converting a model into the HEF format supported by Hailo8L. If you are using Hailo8, please change "Hailo8L" to "Hailo8".
# Your host computer at least have 32GB RAM.

! cd hailo_model_zoo/ && python hailo_model_zoo/main.py compile yolov8n --ckpt best.onnx --hw-arch hailo8l  --calib-path /home/jiahao/datasets/train/images  --classes 3 --performance

<Hailo Model Zoo INFO> Start run for network yolov8n ...
<Hailo Model Zoo INFO> Initializing the hailo8l runner...
[info] Translation started on ONNX model yolov8n
[info] Restored ONNX model yolov8n (completion time: 00:00:00.06)
[info] Extracted ONNXRuntime meta-data for Hailo model (completion time: 00:00:00.22)
[info] NMS structure of yolov8 (or equivalent architecture) was detected.
[info] In order to use HailoRT post-processing capabilities, these end node names should be used: /model.22/cv2.0/cv2.0.2/Conv /model.22/cv3.0/cv3.0.2/Conv /model.22/cv2.1/cv2.1.2/Conv /model.22/cv3.1/cv3.1.2/Conv /model.22/cv2.2/cv2.2.2/Conv /model.22/cv3.2/cv3.2.2/Conv.
[info] Start nodes mapped from original model: 'images': 'yolov8n/input_layer1'.
[info] End nodes mapped from original model: '/model.22/cv2.0/cv2.0.2/Conv', '/model.22/cv3.0/cv3.0.2/Conv', '/model.22/cv2.1/cv2.1.2/Conv', '/model.22/cv3.1/cv3.1.2/Conv', '/model.22/cv2.2/cv2.2.2/Conv', '/model.22/cv3.2/cv3.2.2/Conv'.
[info] Translation 

In [ ]:
# Move yolov8n.hef to /models/Chapter5

! mv   ./hailo_model_zoo/yolov8n.hef ../../models/Chapter5/